In [1]:
# Install library (Uncomment jika jalan di Kaggle/Colab)
!pip install python-terrier pyarabic deep-translator pandas

import pandas as pd
import pyterrier as pt
import os
import sys

# Tambahkan path ke folder src agar modul bisa dibaca
# Sesuaikan path ini dengan lokasi folder Anda
sys.path.append('../src') 

# Import modul buatan sendiri
# Pastikan file src/preprocessing.py dan src/translation.py sudah dibuat dulu!
from preprocessing import preprocess_pipeline
from translation import QueryTranslator

# Inisialisasi PyTerrier
if not pt.started():
    pt.init()

Defaulting to user installation because normal site-packages is not writeable


/tmp/ipykernel_1000866/405323351.py:19: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():
Java started and loaded: pyterrier.java.colab, pyterrier.java, pyterrier.java.24, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/tmp/ipykernel_1000866/405323351.py:20: DeprecationWarning: Call to deprecated method pt.init(). Deprecated since version 0.11.0.
java is now started automatically with default settings. To force initialisation early, run:
pt.java.init() # optional, forces java initialisation
  pt.init()


# Load and Preprocess Data

In [2]:
# Konfigurasi Path
DATA_PATH = '../data'
RAW_DOCS = os.path.join(DATA_PATH, 'raw/fathul_muin.csv')
QUERY_INDO = os.path.join(DATA_PATH, 'queries/queries_indo.csv')
QUERY_ARAB = os.path.join(DATA_PATH, 'queries/queries_arab.csv')

# 1. Load & Preprocess Dokumen
if not os.path.exists(RAW_DOCS):
    print(f"Error: File {RAW_DOCS} tidak ditemukan.")
else:
    print("Loading documents...")
    df_docs = pd.read_csv(RAW_DOCS)
    
    # Terapkan preprocessing pipeline
    print("Preprocessing documents (Normalize + Light Stemming)...")
    df_docs['text_processed'] = df_docs['text_arabic'].apply(preprocess_pipeline)
    print(f"Selesai. {len(df_docs)} dokumen diproses.")
    
    # Intip hasil
    print(df_docs[['text_arabic', 'text_processed']].head(2))

Loading documents...
Preprocessing documents (Normalize + Light Stemming)...
Selesai. 639 dokumen diproses.
                                         text_arabic  \
0   \n | مقدمة\n ...\n بسم الله الرحمن الرحيم\n  ...   
1   الحمد لله الذي هدانا لهذا وما كنا لنهتدي لولا...   

                                      text_processed  
0  | مقدم ... بسم له رحمن رحيم حمد لله تاح جواد م...  
1  حمد لله ذي هدا هذا وما كنا نهتد ولا ان هدا له ...  


# Translasi Kueri

In [3]:
# 2. Translasi & Preprocess Query
if os.path.exists(QUERY_ARAB):
    print("Loading translated queries from disk...")
    df_queries_arab = pd.read_csv(QUERY_ARAB)
else:
    print("File translasi belum ada. Memulai translasi...")
    df_queries = pd.read_csv(QUERY_INDO)
    
    translator = QueryTranslator()
    
    # Translate
    df_queries['query_arab'] = df_queries['query'].apply(translator.translate)
    
    # Preprocess hasil translasi (PENTING: Harus sama perlakuannya dengan dokumen)
    df_queries['query_arab'] = df_queries['query_arab'].apply(preprocess_pipeline)
    
    # Simpan agar tidak perlu translate ulang nanti
    df_queries_arab = df_queries[['qid', 'query', 'query_arab']]
    df_queries_arab.to_csv(QUERY_ARAB, index=False)
    print("Translasi selesai dan disimpan.")

print(df_queries_arab.head())

Loading translated queries from disk...
   qid                                        query  \
0    1                           waktu sholat ashar   
1    2                                  sumpah ila'   
2    3                menyentuh anjing di dalam air   
3    4                         banyak syarat sholat   
4    5  syarat sholat bagi orang yang selalu hadats   

                      query_arab  
0                    وقت صلا عصر  
1                           اقسم  
2                 لمس كلب في ماء  
3                  متطلب صلا ثير  
4  متطلب صلا لاشخاص ذين دي دايما  


In [4]:
# 3. Indexing

# First, check what columns exist in df_docs
print("Checking df_docs structure...")
print(f"Columns in df_docs: {df_docs.columns.tolist()}")
print(f"\nFirst few rows:")
print(df_docs.head())
print(f"\nData shape: {df_docs.shape}")

# Strip any whitespace from column names
df_docs.columns = df_docs.columns.str.strip()

# Check and rename columns if needed
required_columns = {'docno', 'text_processed'}
available_columns = set(df_docs.columns)

print(f"\nRequired columns: {required_columns}")
print(f"Available columns: {available_columns}")
print(f"Missing columns: {required_columns - available_columns}")

# Common column name variations and their mappings
column_mappings = {
    'docno': ['doc_id', 'id', 'document_id', 'doc_no', 'docid'],
    'text_processed': ['text', 'content', 'processed_text', 'text_arab', 'content_processed']
}

# Try to find and rename columns
for target_col, possible_names in column_mappings.items():
    if target_col not in df_docs.columns:
        for possible_name in possible_names:
            if possible_name in df_docs.columns:
                print(f"Renaming '{possible_name}' → '{target_col}'")
                df_docs.rename(columns={possible_name: target_col}, inplace=True)
                break

# Verify we have required columns now
if 'docno' not in df_docs.columns or 'text_processed' not in df_docs.columns:
    raise ValueError(
        f"Missing required columns!\n"
        f"Required: ['docno', 'text_processed']\n"
        f"Available: {df_docs.columns.tolist()}\n"
        f"Please ensure your DataFrame has these columns."
    )

# Setup Index directory
index_dir = os.path.join(DATA_PATH, 'indices/scenario_1_sparse')
os.makedirs(index_dir, exist_ok=True)

# Setup Indexer
indexer = pt.IterDictIndexer(
    index_dir, 
    meta={'docno': 20, 'text': 4096}, 
    overwrite=True,
    stemmer=None,     # Matikan stemmer bawaan (Porter)
    stopwords=None,   # Stopwords sudah dihandle di preprocessing
    tokeniser="UTFTokeniser" 
)

# Jalankan Indexing
print("\nMembuat Index...")
# Note: PyTerrier expects 'text' not 'text_processed' in the dict
docs_to_index = df_docs[['docno', 'text_processed']].copy()
docs_to_index.rename(columns={'text_processed': 'text'}, inplace=True)

index_ref = indexer.index(docs_to_index.to_dict(orient='records'))
print(f"Index tersimpan di: {index_dir}")

# Verify index
index = pt.IndexFactory.of(index_ref)
print(f"\nIndex statistics:")
print(f"Number of documents: {index.getCollectionStatistics().getNumberOfDocuments()}")
print(f"Number of terms: {index.getCollectionStatistics().getNumberOfUniqueTerms()}")
print(f"Number of tokens: {index.getCollectionStatistics().getNumberOfTokens()}")

Checking df_docs structure...
Columns in df_docs: ['doc_no', 'text_arabic', 'text_processed']

First few rows:
         doc_no                                        text_arabic  \
0  Page_V01P031   \n | مقدمة\n ...\n بسم الله الرحمن الرحيم\n  ...   
1  Page_V01P032   الحمد لله الذي هدانا لهذا وما كنا لنهتدي لولا...   
2  Page_V01P033   وعلى آله\n  والرسول من البشر ذكر حر أوحى إليه...   
3  Page_V01P034   وصحبه الفائزين برضا الله.\n وبعد فهذا مختصر ف...   
4  Page_V01P035   وسميته ب قرة العين بمهمات الدين راجيا من الرح...   

                                      text_processed  
0  | مقدم ... بسم له رحمن رحيم حمد لله تاح جواد م...  
1  حمد لله ذي هدا هذا وما كنا نهتد ولا ان هدا له ...  
2  علا اله الرسول من بشر ذكر حر اوحا يه شرع امر ت...  
3  صحب ايز رضا له. بعد هذا مختصر في فقه علا مذهب ...  
4  سميت ب قرة عين مهم دين راجيا من رحمن ان ينتفع ...  

Data shape: (639, 3)

Required columns: {'docno', 'text_processed'}
Available columns: {'text_arabic', 'doc_no', 'text_processed'}
Missin

Index tersimpan di: ../data/indices/scenario_1_sparse

Index statistics:
Number of documents: 639
Number of terms: 11722
Number of tokens: 108610


# Indexing

In [5]:
# # 3. Indexing
# index_dir = os.path.join(DATA_PATH, 'indices/scenario_1_sparse')

# # Setup Indexer
# indexer = pt.IterDictIndexer(
#     index_dir, 
#     meta={'doc_no': 20, 'text': 4096}, 
#     overwrite=True,
#     stemmer=None,     # Matikan stemmer bawaan (Porter)
#     stopwords=None,   # Stopwords sudah dihandle di preprocessing
#     tokeniser="UTFTokeniser" 
# )

# # Jalankan Indexing
# print("Membuat Index...")
# index_ref = indexer.index(df_docs[['doc_no', 'text_processed']].to_dict(orient='records'))
# print(f"Index tersimpan di: {index_dir}")

# Experiment and Evaluation

In [6]:
# # 4. Retrieval & Evaluasi
# QRELS_PATH = os.path.join(DATA_PATH, 'queries/qrels.csv')

# # Load Gold Standard
# qrels = pd.read_csv(QRELS_PATH)
# qrels['label'] = qrels['label'].astype(int)
# qrels['qid'] = qrels['qid'].astype(str)

# # Siapkan Topics
# topics = df_queries_arab[['qid', 'query_arab']].rename(columns={'query_arab': 'query'})
# topics['qid'] = topics['qid'].astype(str)

# # Definisi Model
# bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", controls={"bm25.k_1": 1.2, "bm25.b": 0.75})
# rm3 = bm25 >> pt.rewrite.RM3(index_ref) >> bm25

# # Jalankan Eksperimen
# print("Running PyTerrier Experiment...")
# pipeline = pt.Experiment(
#     [bm25, rm3],
#     topics,
#     qrels,
#     eval_metrics=["map", "recip_rank", "recall_10", "recall_20"],
#     names=["BM25 (Baseline)", "BM25 + RM3"]
# )

# print(pipeline)
# # pipeline.to_csv('hasil_skenario_1.csv')

In [7]:
# 4. Retrieval & Evaluasi
QRELS_PATH = os.path.join(DATA_PATH, 'queries/qrels.csv')

# Load Gold Standard
qrels = pd.read_csv(QRELS_PATH)

# ===== CHECK AND FIX QRELS COLUMNS =====
print("Checking qrels structure...")
print(f"Columns in qrels: {qrels.columns.tolist()}")
print(f"\nSample qrels:")
print(qrels.head())

# Strip whitespace from column names
qrels.columns = qrels.columns.str.strip()

# Rename columns to match PyTerrier expectations
# PyTerrier expects: qid, docno, label (NOT query_id, doc_id, relevance)
column_rename_map = {}

# Handle doc ID column
if 'doc_no' in qrels.columns:
    column_rename_map['doc_no'] = 'docno'
elif 'doc_id' in qrels.columns:
    column_rename_map['doc_id'] = 'docno'

# Apply renaming
if column_rename_map:
    qrels.rename(columns=column_rename_map, inplace=True)

print(f"\nRenamed columns: {column_rename_map}")
print(f"Final qrels columns: {qrels.columns.tolist()}")

# Convert data types
qrels['label'] = qrels['label'].astype(int)
qrels['qid'] = qrels['qid'].astype(str)
qrels['docno'] = qrels['docno'].astype(str)

print(f"\nFinal qrels structure:")
print(qrels.head())
print(f"Shape: {qrels.shape}")
# ===== END OF FIX =====

# Siapkan Topics
topics = df_queries_arab[['qid', 'query_arab']].rename(columns={'query_arab': 'query'})
topics['qid'] = topics['qid'].astype(str)

print(f"\nTopics structure:")
print(topics.head())
print(f"Shape: {topics.shape}")

# Definisi Model
print("\nSetting up retrieval models...")
bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", controls={"bm25.k_1": 1.2, "bm25.b": 0.75})
rm3 = bm25 >> pt.rewrite.RM3(index_ref) >> bm25

# Jalankan Eksperimen
print("\nRunning PyTerrier Experiment...")
pipeline = pt.Experiment(
    [bm25, rm3],
    topics,
    qrels,
    eval_metrics=["map", "recip_rank", "recall_10", "recall_20", "recall_100"],
    names=["BM25 (Baseline)", "BM25 + RM3"]
)

print("\n=== Evaluation Results ===")
print(pipeline)

# Save results
output_dir = os.path.join(DATA_PATH, 'results')
os.makedirs(output_dir, exist_ok=True)
pipeline.to_csv(os.path.join(output_dir, 'hasil_skenario_1.csv'))
print(f"\nResults saved to: {output_dir}/hasil_skenario_1.csv")
# ```

# ## Key Fix:

# The issue was I was renaming to `query_id` and `doc_id`, but PyTerrier actually expects `qid` and `docno` to match with the topics DataFrame.

# **Correct column names:**
# - Topics: `qid`, `query`
# - Qrels: `qid`, `docno`, `label`

# **Expected qrels format after fix:**
# ```
# qid, docno, label
# 1, PageV01P087, 1
# 2, PageV01P522, 1

Checking qrels structure...
Columns in qrels: ['qid', 'doc_no', 'label']

Sample qrels:
   qid        doc_no  label
0    1  Page_V01P087      1
1    2  Page_V01P522      1
2    3  Page_V01P080      1
3    4  Page_V01P040      1
4    5  Page_V01P045      1

Renamed columns: {'doc_no': 'docno'}
Final qrels columns: ['qid', 'docno', 'label']

Final qrels structure:
  qid         docno  label
0   1  Page_V01P087      1
1   2  Page_V01P522      1
2   3  Page_V01P080      1
3   4  Page_V01P040      1
4   5  Page_V01P045      1
Shape: (10, 3)

Topics structure:
  qid                          query
0   1                    وقت صلا عصر
1   2                           اقسم
2   3                 لمس كلب في ماء
3   4                  متطلب صلا ثير
4   5  متطلب صلا لاشخاص ذين دي دايما
Shape: (10, 2)

Setting up retrieval models...

Running PyTerrier Experiment...


/tmp/ipykernel_1000866/3367069010.py:53: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25", controls={"bm25.k_1": 1.2, "bm25.b": 0.75})



=== Evaluation Results ===
              name       map  recip_rank  recall_10  recall_20  recall_100
0  BM25 (Baseline)  0.455823    0.455823        0.5        0.5         0.7
1       BM25 + RM3  0.453478    0.453478        0.5        0.5         0.7

Results saved to: ../data/results/hasil_skenario_1.csv


In [8]:
# ===== DIAGNOSTIC CODE =====
print("=== COMPREHENSIVE DIAGNOSIS ===\n")

# 1. Check if documents were indexed correctly
print("1. INDEX CHECK:")
index = pt.IndexFactory.of(index_ref)
print(f"   Documents in index: {index.getCollectionStatistics().getNumberOfDocuments()}")
print(f"   Unique terms: {index.getCollectionStatistics().getNumberOfUniqueTerms()}")
print(f"   Total tokens: {index.getCollectionStatistics().getNumberOfTokens()}")

# 2. Check docno format consistency
print("\n2. DOCNO FORMAT CHECK:")
print(f"   Indexed docs sample docnos:")
lexicon = index.getLexicon()
docs_in_index = []
for i in range(min(10, index.getCollectionStatistics().getNumberOfDocuments())):
    meta = index.getMetaIndex()
    docno = meta.getItem("docno", i)
    docs_in_index.append(docno)
    print(f"     {docno}")

print(f"\n   Qrels docnos sample:")
print(qrels['docno'].head(10).tolist())

print(f"\n   Topics qids sample:")
print(topics['qid'].head(10).tolist())

# 3. Check if there's a mismatch
print("\n3. MATCHING CHECK:")
qrels_docnos = set(qrels['docno'].unique())
index_docnos = set(docs_in_index)  # Sample only, but should give us a clue

print(f"   Unique docnos in qrels: {len(qrels_docnos)}")
print(f"   Sample docnos in index: {len(index_docnos)}")
print(f"   Any overlap in sample? {bool(qrels_docnos.intersection(index_docnos))}")

if not qrels_docnos.intersection(index_docnos):
    print("\n   ⚠️ NO OVERLAP DETECTED!")
    print(f"   Qrels format: {list(qrels_docnos)[:5]}")
    print(f"   Index format: {list(index_docnos)[:5]}")

# 4. Run a test retrieval
print("\n4. TEST RETRIEVAL:")
test_results = bm25.transform(topics.head(3))
print(f"   Retrieved {len(test_results)} results")
if len(test_results) > 0:
    print(f"   Sample results:")
    print(test_results.head(10)[['qid', 'docno', 'score', 'rank']])
else:
    print("   ⚠️ NO RESULTS RETRIEVED!")

# 5. Check query processing
print("\n5. QUERY CHECK:")
print("   Sample queries after translation:")
for idx, row in topics.head(3).iterrows():
    print(f"   Query {row['qid']}: '{row['query']}'")
    print(f"   Length: {len(row['query'])} chars, {len(row['query'].split())} words")

# 6. Check document processing
print("\n6. DOCUMENT CHECK:")
print("   Sample processed documents:")
for idx in range(min(3, len(df_docs))):
    print(f"   Doc {df_docs.iloc[idx]['docno']}:")
    text = df_docs.iloc[idx]['text_processed']
    print(f"   Length: {len(text)} chars, {len(text.split())} words")
    print(f"   First 100 chars: {text[:100]}")

# 7. Check for exact docno match
print("\n7. EXACT DOCNO COMPARISON:")
qrel_sample = qrels['docno'].iloc[0]
print(f"   First qrel docno: '{qrel_sample}' (type: {type(qrel_sample)})")

# Try to find it in df_docs
if 'docno' in df_docs.columns:
    matching_docs = df_docs[df_docs['docno'] == qrel_sample]
    print(f"   Found in df_docs? {len(matching_docs) > 0}")
    if len(matching_docs) > 0:
        print(f"   Matching doc: {matching_docs['docno'].iloc[0]}")
    else:
        print(f"   Searching for similar docnos in df_docs:")
        similar = df_docs[df_docs['docno'].str.contains(qrel_sample[:8], na=False)]['docno'].head(5)
        print(f"   Similar docnos: {similar.tolist()}")

=== COMPREHENSIVE DIAGNOSIS ===

1. INDEX CHECK:
   Documents in index: 639
   Unique terms: 11722
   Total tokens: 108610

2. DOCNO FORMAT CHECK:
   Indexed docs sample docnos:
     Page_V01P031
     Page_V01P032
     Page_V01P033
     Page_V01P034
     Page_V01P035
     Page_V01P036
     Page_V01P037
     Page_V01P038
     Page_V01P039
     Page_V01P040

   Qrels docnos sample:
['Page_V01P087', 'Page_V01P522', 'Page_V01P080', 'Page_V01P040', 'Page_V01P045', 'Page_V01P209', 'Page_V01P142', 'Page_V01P063', 'Page_V01P243', 'Page_V01P452']

   Topics qids sample:
['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

3. MATCHING CHECK:
   Unique docnos in qrels: 10
   Sample docnos in index: 10
   Any overlap in sample? True

4. TEST RETRIEVAL:
   Retrieved 837 results
   Sample results:
  qid         docno      score  rank
0   1  Page_V01P087  14.783294     0
1   1  Page_V01P089  12.971664     1
2   1  Page_V01P172   9.748289     2
3   1  Page_V01P212   9.701254     3
4   1  Page_V01P081 

In [9]:
print("=== COMPLETE DIAGNOSTIC FOR 90% RECALL@700 ===\n")

# 1. Check qrels coverage
qrels_docs = set(qrels['docno'].unique())
indexed_docs = set(df_docs['docno'].unique())
missing_docs = qrels_docs - indexed_docs

print(f"1. Document Coverage:")
print(f"   Unique relevant docs in qrels: {len(qrels_docs)}")
print(f"   Documents in index: {len(indexed_docs)}")
print(f"   Missing from index: {len(missing_docs)}")
if missing_docs:
    print(f"   Missing docs: {missing_docs}")

# 2. Run retrieval to find what's not retrieved
print(f"\n2. Retrieval Coverage:")
results_all = bm25.transform(topics)

total_relevant = len(qrels)
retrieved_relevant = 0
not_retrieved = []

for qid in topics['qid']:
    rel_docs = set(qrels[qrels['qid'] == qid]['docno'])
    retrieved_docs = set(results_all[results_all['qid'] == qid]['docno'])
    
    found = rel_docs.intersection(retrieved_docs)
    missing = rel_docs - retrieved_docs
    
    retrieved_relevant += len(found)
    
    if missing:
        for doc in missing:
            not_retrieved.append({'qid': qid, 'docno': doc})
            query = topics[topics['qid'] == qid]['query'].iloc[0]
            print(f"\n   Query {qid}: '{query}'")
            print(f"   Missing doc: {doc}")
            
            if doc in df_docs['docno'].values:
                doc_text = df_docs[df_docs['docno'] == doc]['text_processed'].iloc[0]
                print(f"   Doc length: {len(doc_text)} chars")
                print(f"   Doc preview: '{doc_text[:150]}'")
            else:
                print(f"   ⚠️ Doc not in df_docs!")

print(f"\n3. Summary:")
print(f"   Total relevant judgments: {total_relevant}")
print(f"   Retrieved relevant: {retrieved_relevant}")
print(f"   Not retrieved: {len(not_retrieved)}")
print(f"   Recall: {retrieved_relevant/total_relevant:.2%}")

=== COMPLETE DIAGNOSTIC FOR 90% RECALL@700 ===

1. Document Coverage:
   Unique relevant docs in qrels: 10
   Documents in index: 639
   Missing from index: 0

2. Retrieval Coverage:

   Query 2: 'اقسم'
   Missing doc: Page_V01P522
   Doc length: 704 chars
   Doc preview: 'ولو تزوج مفارقت دون ثلاث ولو بعد زوج اخر عادت بقيته. ولو ادعا رجع عدة وهي منقض ولم تنكح فان اتفقا علا وقت انقضاء يوم جمع قال: راجعت قبل قالت بل عده حل'

3. Summary:
   Total relevant judgments: 10
   Retrieved relevant: 9
   Not retrieved: 1
   Recall: 90.00%


In [10]:
print("=== QUERY RETRIEVAL SUCCESS REPORT ===\n")

# Get all retrieval results
results_all = bm25.transform(topics)

# Create a detailed report
report_data = []

for idx, row in topics.iterrows():
    qid = row['qid']
    query = row['query']
    
    # Get relevant documents for this query
    rel_docs = qrels[qrels['qid'] == qid]['docno'].tolist()
    
    # Get retrieved documents for this query
    retrieved = results_all[results_all['qid'] == qid]
    retrieved_docs = retrieved['docno'].tolist()
    
    # Check which relevant docs were found
    found_docs = [doc for doc in rel_docs if doc in retrieved_docs]
    missing_docs = [doc for doc in rel_docs if doc not in retrieved_docs]
    
    # Get ranks of found docs
    ranks = []
    for doc in found_docs:
        rank = retrieved[retrieved['docno'] == doc]['rank'].iloc[0]
        ranks.append(rank)
    
    # Calculate metrics
    total_relevant = len(rel_docs)
    total_found = len(found_docs)
    success_rate = (total_found / total_relevant * 100) if total_relevant > 0 else 0
    
    # Store report data
    report_data.append({
        'qid': qid,
        'query': query,
        'total_relevant': total_relevant,
        'found': total_found,
        'missing': len(missing_docs),
        'success_rate': success_rate,
        'best_rank': min(ranks) if ranks else 'N/A',
        'worst_rank': max(ranks) if ranks else 'N/A',
        'found_docs': found_docs,
        'missing_docs': missing_docs
    })

# Create DataFrame for easy viewing
report_df = pd.DataFrame(report_data)

# Display summary table
print("=" * 100)
print(f"{'QID':<5} {'Query':<25} {'Relevant':<10} {'Found':<8} {'Missing':<8} {'Success %':<12} {'Best Rank':<10} {'Worst Rank':<10}")
print("=" * 100)

for _, row in report_df.iterrows():
    print(f"{row['qid']:<5} {row['query']:<25} {row['total_relevant']:<10} {row['found']:<8} {row['missing']:<8} {row['success_rate']:<12.1f} {str(row['best_rank']):<10} {str(row['worst_rank']):<10}")

print("=" * 100)

# Overall statistics
print(f"\n{'OVERALL STATISTICS':^100}")
print("=" * 100)
print(f"Total queries: {len(report_df)}")
print(f"Queries with 100% success: {len(report_df[report_df['success_rate'] == 100])}")
print(f"Queries with partial success: {len(report_df[(report_df['success_rate'] > 0) & (report_df['success_rate'] < 100)])}")
print(f"Queries with 0% success: {len(report_df[report_df['success_rate'] == 0])}")
print(f"Average success rate: {report_df['success_rate'].mean():.2f}%")
print(f"Total relevant documents: {report_df['total_relevant'].sum()}")
print(f"Total found: {report_df['found'].sum()}")
print(f"Total missing: {report_df['missing'].sum()}")
print(f"Overall recall: {(report_df['found'].sum() / report_df['total_relevant'].sum() * 100):.2f}%")

# Detailed breakdown for each query
print(f"\n\n{'DETAILED QUERY BREAKDOWN':^100}")
print("=" * 100)

for _, row in report_df.iterrows():
    print(f"\n📋 Query {row['qid']}: '{row['query']}'")
    print(f"   Status: {row['found']}/{row['total_relevant']} relevant docs found ({row['success_rate']:.1f}%)")
    
    if row['found_docs']:
        print(f"   ✓ Found documents:")
        retrieved_q = results_all[results_all['qid'] == row['qid']]
        for doc in row['found_docs']:
            rank = retrieved_q[retrieved_q['docno'] == doc]['rank'].iloc[0]
            score = retrieved_q[retrieved_q['docno'] == doc]['score'].iloc[0]
            print(f"      • {doc} (Rank: {rank}, Score: {score:.4f})")
    
    if row['missing_docs']:
        print(f"   ✗ Missing documents:")
        for doc in row['missing_docs']:
            print(f"      • {doc} (NOT RETRIEVED)")

print("\n" + "=" * 100)

# Save to CSV for further analysis
report_df_simple = report_df[['qid', 'query', 'total_relevant', 'found', 'missing', 'success_rate', 'best_rank', 'worst_rank']]
output_file = os.path.join(output_dir, 'query_success_report.csv')
report_df_simple.to_csv(output_file, index=False)
print(f"\n📊 Report saved to: {output_file}")

# Identify problematic queries
print(f"\n\n{'PROBLEMATIC QUERIES (for investigation)':^100}")
print("=" * 100)

problematic = report_df[report_df['success_rate'] < 100]
if len(problematic) > 0:
    for _, row in problematic.iterrows():
        print(f"\n⚠️  Query {row['qid']}: '{row['query']}'")
        print(f"    Success: {row['success_rate']:.1f}%")
        print(f"    Missing: {row['missing_docs']}")
else:
    print("✓ No problematic queries - all queries found all relevant documents!")
# Add emoji indicators
report_df['status'] = report_df['success_rate'].apply(
    lambda x: '✓ Perfect' if x == 100 
    else '⚠️ Partial' if x > 0 
    else '✗ Failed'
)

=== QUERY RETRIEVAL SUCCESS REPORT ===

QID   Query                     Relevant   Found    Missing  Success %    Best Rank  Worst Rank
1     وقت صلا عصر               1          1        0        100.0        0          0         
2     اقسم                      1          0        1        0.0          N/A        N/A       
3     لمس كلب في ماء            1          1        0        100.0        0          0         
4     متطلب صلا ثير             1          1        0        100.0        57         57        
5     متطلب صلا لاشخاص ذين دي دايما 1          1        0        100.0        161        161       
6     سنة قراء سور كهف          1          1        0        100.0        0          0         
7     اشياء تي تبطل صلا         1          1        0        100.0        1          1         
8     اشياء محرم علا اشخاص مصاب الحدث 1          1        0        100.0        0          0         
9     هل يمكن لد ان يبطل تزام زكاة؟ 1          1        0        100.0        382     